# Train an IntegratedML model on Marketing Campaign Dataset
## Use JDBC to connect to InterSystems IRIS database
This Notebook demonstrates:
- Using the JayDeBeApi Python library to connect to InterSystems IRIS
- Creating views to segment data into training and test sets
- Defining and training an IntegratedML model to predict marketing campaign responses
- Comparing the resulting model's predictions to data in the test set (that the model was not trained on)
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics on the test set data

In [1]:
!pip install jupyterthemes

In [2]:
!jt -r

Reset css and font defaults in:
/root/.jupyter/custom &
/root/.local/share/jupyter/nbextensions


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

!ls /tf/intersystems-jdbc-3.1.0.jar
RUN apt-get update && \
    DEBIAN_FRONTEND=noninteractive \
    apt-get -y install default-jre-headless && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

In [2]:
!pip install JayDeBeApi

In [3]:
!pip list JPype1

Package                Version
---------------------- -----------
absl-py                0.9.0
asn1crypto             0.24.0
astunparse             1.6.3
attrs                  19.3.0
backcall               0.1.0
bleach                 3.1.5
cachetools             4.1.0
certifi                2020.4.5.1
chardet                3.0.4
cryptography           2.1.4
cycler                 0.10.0
decorator              4.4.2
defusedxml             0.6.0
entrypoints            0.3
gast                   0.3.3
google-auth            1.14.2
google-auth-oauthlib   0.4.1
google-pasta           0.2.0
grpcio                 1.28.1
h5py                   2.10.0
idna                   2.6
importlib-metadata     1.6.0
ipykernel              5.1.1
ipython                7.14.0
ipython-genutils       0.2.0
ipywidgets             7.5.1
JayDeBeApi             1.2.3
jedi                   0.17.0
Jinja2                 2.11.2
joblib                 0.15.1
JPype1                 0.7.5
jsonschema             3

### 1. Set environment variables, if necessary

In [ ]:
#import os
#os.environ['JAVA_HOME']='C:\Progra~1\Java\jdk1.8.0_241'
#os.environ['CLASSPATH'] = 'C:\interSystems\IRIS20194\dev\java\lib\JDK18\intersystems-jdbc-3.0.0.jar'
#os.environ['HADOOP_HOME']='C:\hadoop\bin'  #winutil binary must be in Hadoop's Home

### 2. Get jdbc connection and cursor

In [4]:

import jaydebeapi
url = "jdbc:IRIS://irisimlsvr:51773/USER" #"jdbc:IRIS://172.17.0.1:8091/USER"
driver = 'com.intersystems.jdbc.IRISDriver'
user = "SUPERUSER"
password = "SYS"
#libx = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18"
#jarfile = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18/intersystems-jdbc-3.0.0.jar"
jarfile = "./intersystems-jdbc-3.1.0.jar"

In [5]:
conn = jaydebeapi.connect(driver, url, [user, password], jarfile)
curs = conn.cursor()

### 3. specify the source data table

In [6]:
dataTable = 'SQLUser.Campaign'

### 4. Execute a query and display results in Pandas DataFrame

In [7]:
import pandas as pd
from IPython.display import display

df = pd.read_sql("select TOP 20 * from %s" % dataTable, conn)
display(df)

,AGE,AMOUNT,COUNTRY_OF_BIRTH,EDUCATION,EMPLOYMENT,GENDER,INV_EARNINGS,INV_LOSSES,MARITAL_STATUS,OCCUPATION,PROSPECT_ID,RACE,RELATIONSHIP,RESPONSE,WORKWEEK_LENGTH,YEARS_EDUC
0,50,123.90,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,REPAIRCRAFT,77001107,WHITE,HUSBAND,0,40,9
1,49,97.91,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,LOGISTICSTRANSP,77001140,WHITE,HUSBAND,0,40,9
2,37,135.65,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,REPAIRCRAFT,77001143,WHITE,HUSBAND,0,40,9
3,43,93.23,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,CLEANINGHANDLING,77001187,WHITE,HUSBAND,0,40,9
4,54,143.72,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,LOGISTICSTRANSP,77001412,WHITE,HUSBAND,0,40,9
5,27,232.20,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,SERVICEVARIOUS,77001429,WHITE,HUSBAND,0,40,9
6,29,103.46,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,REPAIRCRAFT,77001443,WHITE,HUSBAND,0,40,9
7,29,123.53,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,ADMINOFFICE,77001484,WHITE,HUSBAND,0,40,9
8,21,230.07,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,MACHINEOPERINSP,77001526,WHITE,HUSBAND,0,40,9
9,36,48.95,USA,HIGHSCHOOL,BUSINESS,MALE,0,0,MARRIEDCIVSPO,MACHINEOPERINSP,77001634,WHITE,HUSBAND,0,40,9


In [8]:
# Show number rows
df1 = pd.read_sql("SELECT COUNT(*) FROM %s" % dataTable, conn)
display(df1)

,Aggregate_1
0,48842


### 5. Make some views to split training and testing datasets

In [9]:
# Small training set view
curs.execute("CREATE VIEW SQLUser.CampaignTrainingSmall AS SELECT * FROM SQLUser.Campaign WHERE ID<3001")
# Small prediction set
curs.execute("CREATE VIEW SQLUser.CampaignPredictSmall AS SELECT * FROM SQLUser.Campaign WHERE ID>47000")

DatabaseError: java.sql.SQLException: [SQLCODE: <-201>:<Table or view name not unique>]
[Location: <ServerLoop>]

### 6. Create and Train an IntegratedML Model using default settings
IntegratedML only needs a model name, the name of the column that is the target column to predict, and a table (or SELECT query to specify input columns.

In [10]:
curs.execute("CREATE MODEL CampaignModel PREDICTING (RESPONSE) FROM CampaignTrainingSmall")

DatabaseError: java.sql.SQLException: [SQLCODE: <-180>:<Model name not unique>]
[Location: <ServerLoop>]
[%msg: <Model name 'CampaignModel' is not unique>]

Now that the model is defined, you can TRAIN it, which invokes the AutoML machine learning procedure.

In [11]:
curs.execute("TRAIN MODEL CampaignModel")

Once that finishes, you can see some information about the model in the "ML_TRAINED_MODELS" table.

In [12]:
df3 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", conn)
display(df3)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,CampaignModel,CampaignModel2,AutoML,2020-06-29 13:44:07.324000,classification,"ModelType:Logistic Regression, Package:sklearn..."
1,CampaignModel,CampaignModel4,AutoML,2020-06-29 15:25:48.832000,classification,"ModelType:Logistic Regression, Package:sklearn..."


### 7. Compare model output to data it has not seen yet
Now you can use SQL to SELECT data from another table, run the IntegratedML model on this new data, and see how well the predictions match the data!

In [17]:
df4 = pd.read_sql("SELECT PREDICT(CampaignModel) AS PredictedResponse, \
                  response AS ActualResponse FROM CampaignPredictSmall", conn)
display(df4)

,PredictedResponse,ActualResponse
0,1,0
1,1,0
2,1,1
3,1,0
4,1,1
...,...,...
1837,1,0
1838,0,0
1839,0,0
1840,0,0


### 8. VALIDATE MODEL command calculates accuracy metrics
You can certainly take that output above and calculate the accuracy using a standard formula, but IntegratedML has a built-in function to do that!

Each time you run the command "VALIDATE MODEL..." it generates a set of metrics calculated on the data passed into the query. Since this table can be a bit difficult to read in its raw form we use a simple "pivot" call to arrange the data.

In [18]:
curs.execute("VALIDATE MODEL CampaignModel FROM SQLUser.CampaignPredictSmall")
df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS", conn)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

METRIC_NAME,Accuracy,F-Measure,Precision,Recall
VALIDATION_RUN_NAME,,,,
CampaignModel21,0.75,0.73,0.68,0.8
